## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("sesac-CH1")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [5]:
# template 정의
template = "{animal}의 다리의 개수는 몇 개인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['animal'], input_types={}, partial_variables={}, template='{animal}의 다리의 개수는 몇 개인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(animal="기린")
prompt

'기린의 다리의 개수는 몇 개인가요?'

In [7]:
# prompt 생성
prompt = prompt_template.format(animal="문어")
prompt

'문어의 다리의 개수는 몇 개인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [9]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{animal}의 다리의 개수는 몇 개인가요?")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [10]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"animal": "기린"}

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='기린의 다리 개수는 4개입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 21, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f12167b370', 'id': 'chatcmpl-C4MqWlfBqOSkOwVYaQrghRGZb8p3P', 'finish_reason': 'stop', 'logprobs': None}, id='run-f99674f4-8722-4908-b0dc-d9c0a87ece74-0', usage_metadata={'input_tokens': 21, 'output_tokens': 13, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

아래는 스트리밍을 출력하는 예시 입니다.

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

기린의 다리 개수는 4개입니다.

In [13]:
input = {"animal": "문어"}
chain.invoke(input)

AIMessage(content='문어의 다리 개수는 8개입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 21, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-C4MrmCTcabxRLq0qyEo2p2NzwqSJ8', 'finish_reason': 'stop', 'logprobs': None}, id='run-1028827e-5cd8-44b1-b93f-00943779c6f6-0', usage_metadata={'input_tokens': 21, 'output_tokens': 13, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
answer = chain.stream(input)
stream_response(answer)

문어의 다리 개수는 8개입니다.

### 출력파서(Output Parser)


In [15]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [16]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [17]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"animal": "게"}
chain.invoke(input)

'게의 다리 개수는 보통 10개입니다.'

In [18]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

게의 다리 개수는 10개입니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [21]:
template = """
당신은 유명한 요리사입니다. 요리 메뉴를 말해주면 필요한 재료와 조리법을 알려주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{food}

#FORMAT:
- 요리 이름:
- 재료:
- 조리법:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [22]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [23]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"food": "알리오올리오파스타"}))

- 요리 이름: 알리오 올리오 파스타

- 재료:
  - 스파게티 면 200g
  - 마늘 4~5쪽 (얇게 슬라이스)
  - 올리브 오일 5큰술
  - 소금 적당량
  - 후추 약간
  - 파슬리 (선택 사항, 다진 것)
  - 휘핑치즈 또는 파마산 치즈 (선택 사항)

- 조리법:
  1. 큰 냄비에 물을 끓이고 소금을 넣은 후, 스파게티 면을 포장지에 적힌 시간만큼 삶아주세요. 다 삶은 후 체에 받쳐 물기를 빼줍니다.
  2. 팬에 올리브 오일을 넣고 중불에서 가열합니다.
  3. 올리브 오일이 달궈지면 슬라이스한 마늘을 넣고, 노릇노릇해질 때까지 볶아줍니다. 마늘이 타지 않도록 주의하세요.
  4. 삶은 파스타를 팬에 넣고 고루 섞어줍니다.
  5. 소금과 후추로 간을 맞추고, 원한다면 다진 파슬리와 치즈를 넣어 풍미를 더하세요.
  6. 바로 그릇에 담아 따뜻하게 서빙하세요.


In [24]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"food": "돼지고기김치찌개"})
# 스트리밍 출력
stream_response(answer)

- 요리 이름: 돼지고기 김치찌개  
- 재료:  
  - 돼지고기 (목살 또는 앞다리살) 200g  
  - 숙성 김치 300g  
  - 두부 1/2모 (약 150g)  
  - 대파 1대  
  - 양파 1개  
  - 마늘 3쪽  
  - 고춧가루 1 큰술  
  - 된장 1 작은술 (선택사항)  
  - 물 또는 육수 4컵  
  - 식용유 1 큰술  
  - 소금, 후추 약간  
  
- 조리법:  
  1. 돼지고기는 한입 크기로 썰어 준비하고, 대파는 어슷하게 썰고, 양파는 채 썰어 둡니다.  
  2. 팬에 식용유를 두르고, 마늘 다진 것을 넣어 향을 냅니다.  
  3. 돼지고기를 넣고 겉면이 하얗게 익을 때까지 볶습니다.  
  4. 김치와 고춧가루를 넣고 함께 볶아줍니다.  
  5. 볶은 재료에 물 또는 육수를 붓고 끓입니다.  
  6. 끓기 시작하면 끓는 국물에 된장을 풀어 넣고, 중불로 줄여 10분 정도 끓입니다.  
  7. 두부는 먹기 좋은 크기로 썰어 넣고, 양파와 대파도 넣어줍니다.  
  8. 5~10분 더 끓인 후, 소금과 후추로 간을 맞추고 불을 끕니다.  
  9. 그릇에 담아 따뜻하게 즐기세요!

In [28]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"food": "세상에서 가장 맛있는 요리"})
# 스트리밍 출력
stream_response(answer)

물론입니다! 세상에서 가장 맛있는 요리를 만들어 드리겠습니다. 아래는 예시입니다.

#상황:
세상에서 가장 맛있는 요리

#FORMAT:
- 요리 이름: 트러플 크림 리조또
- 재료:
  - 아르borio 쌀 200g
  - 신선한 트러플 또는 트러플 오일 1큰술
  - 닭 육수 또는 야채 육수 1.5L
  - 버터 50g
  - 양파 1개 (잘게 다진 것)
  - 마늘 2쪽 (다진 것)
  - 건조 화이트 와인 100ml
  - 생크림 100ml
  - 파르메산 치즈 가루 50g
  - 소금, 후추 적당량
- 조리법:
  1. 육수를 냄비에 담아 끓이면서 약불로 유지하세요.
  2. 팬에 버터 30g를 녹이고, 다진 양파를 투명해질 때까지 볶습니다.
  3. 마늘을 넣고 1분간 더 볶아 향을 내세요.
  4. 쌀을 넣고 투명해질 때까지 볶아줍니다.
  5. 화이트 와인을 부어 알코올이 날아가도록 저어가며 끓입니다.
  6. 따뜻한 육수 한 국자씩 떠서 넣으며 계속 저어가며 쌀이 익고 크리미해질 때까지 반복하세요(약 18-20분).
  7. 생크림과 파르메산 치즈를 넣고 잘 섞어 크림소스를 만드세요.
  8. 마지막으로 트러플 오일 몇 방울과 잘게 썬 트러플을 넣어 풍미를 더하세요.
  9. 소금과 후추로 간을 맞추고, 접시에 담아 바로 드세요.

이 요리는 풍부한 트러플 향과 크리미한 식감이 조화를 이루어 세상에서 가장 맛있는 요리 중 하나입니다.

In [29]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"food": "세상에서 가장 맛없는 요리"})
# 스트리밍 출력
stream_response(answer)

- 요리 이름: 실패의 블랙스튜  
- 재료:  
  - 오래된 냉장고에 남은 잡다한 야채 (양파, 당근, 감자, 호박 등)  
  - 유통기한 지난 고기 조각 또는 인스턴트 고기  
  - 물 또는 저렴한 육수 팩  
  - 싱거운 소금과 후추  
  - 지나치게 짜거나 쓴 향신료 (예: 과다한 고수, 계피 가루)  

- 조리법:  
  1. 냄비에 야채를 넣고 중불에서 아무 맛도 가미하지 않고 볶지 않는다.  
  2. 고기 조각을 넣고 함께 볶지 않으며, 그냥 냄비에 내려놓는다.  
  3. 물 또는 육수 팩을 넣고 끓이기 시작한다.  
  4. 끓기 시작하면 소금과 후추, 향신료를 지나치게 많이 넣어 맛을 망친다.  
  5. 너무 강불로 계속 끓이며 재료가 흐물흐물해지고 비린내와 이상한 맛이 나는 상태가 되도록 조리한다.  
  6. 완성된 스튜는 잡내와 무미, 텁텁한 맛이 어우러진 최고의 '세상에서 가장 맛없는 요리'가 된다.

In [26]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"food": "우라늄"})
# 스트리밍 출력
stream_response(answer)

죄송하지만, "우라늄"은 요리 재료로 적합하지 않으며 매우 위험한 물질입니다. 안전하게 즐기실 수 있는 요리 메뉴를 알려드릴 수 있으니, 다른 재료나 요리 아이디어를 알려주시면 도움드리겠습니다.